In [1]:
from engine import Engine

In [3]:
# using the built in run loop
engine = Engine(caption='Built in run loop')

def callback():
    pass

def mouse_pressed():
    print('mouse: {}'.format(engine.mouse))

engine.run(callback=callback, mouse_pressed=mouse_pressed)

Running...
mouse: (344, 348)
mouse: (296, 227)
mouse: (218, 173)


In [4]:
# using a custom run loop

engine = Engine(caption='Custom run loop')

running = True
while running:
    for e in engine.events():
        if engine.quit_event(e):
            running = False
            
    engine.step_physics()
    engine.render()
    engine.clock_tick()
    
engine.close()